# Visualization: Integration Results

This notebook visualizes:
1. Method comparison (MARIO vs MaxFuse)
2. Joint embeddings (CCA, UMAP)
3. Spatial cell type mapping
4. Harmony batch correction

**Prerequisites**: Run `preprocessing.ipynb` and `integration.ipynb` first

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import anndata as ad
import matplotlib.pyplot as plt
from scipy import sparse
from scipy.io import mmread
import warnings
warnings.filterwarnings('ignore')

# Set plotting defaults
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['figure.dpi'] = 100
import pickle
import json
import os

In [ ]:
# Compute UMAP on BOTH embeddings and compare

# 1. Unified embedding (averaged CCA scores) - TRUE integration
sc.pp.neighbors(cca_adata_unified, n_neighbors=15, use_rep='X')
sc.tl.umap(cca_adata_unified, min_dist=0.08, spread=1.2)
5
# 2. Stacked embedding (RNA and Protein separate)
sc.pp.neighbors(cca_adata, n_neighbors=15, use_rep='X')
sc.tl.umap(cca_adata, min_dist=0.08, spread=1.2)

# Plot comparison
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Unified embedding - colored by a marker (no modality split)
cd45_expr = np.zeros(len(unified_embedding))
if 'PTPRC' in rna_adata_lognorm.var_names:
    cd45_idx = list(rna_adata_lognorm.var_names).index('PTPRC')
    cd45_vals = rna_adata_lognorm.X[:, cd45_idx]
    if sparse.issparse(cd45_vals):
        cd45_vals = cd45_vals.toarray().flatten()
    cd45_expr = cd45_vals[aligned_rna_indices]
cca_adata_unified.obs['CD45_RNA'] = cd45_expr

sc.pl.umap(cca_adata_unified, color='CD45_RNA', ax=axes[0], show=False,
           title='Unified Embedding (Averaged CCA)\nColored by CD45 RNA', cmap='viridis')

# Stacked embedding - colored by modality
sc.pl.umap(cca_adata, color='modality', ax=axes[1], show=False,
           title='Stacked Embedding\n(RNA and Protein separate)')

# Stacked embedding with connecting lines - draw scatter FIRST, then lines ON TOP
sc.pl.umap(cca_adata, color='modality', ax=axes[2], show=False,
           title='Stacked with Matched Pairs Connected')

# Now draw connecting lines ON TOP of the scatter
umap_coords = cca_adata.obsm['X_umap']
n_pairs = len(rna_cca_scores)
n_show_pairs = 200
np.random.seed(42)
pair_sample = np.random.choice(n_pairs, min(n_show_pairs, n_pairs), replace=False)

for pair_idx in pair_sample:
    rna_umap = umap_coords[pair_idx]
    prot_umap = umap_coords[n_pairs + pair_idx]
    # Draw lines with higher visibility: thicker, darker, on top
    axes[2].plot([rna_umap[0], prot_umap[0]], [rna_umap[1], prot_umap[1]], 
                 color='black', alpha=0.4, linewidth=1.0, zorder=10)

axes[2].set_title(f'Matched Pairs Connected (n={n_show_pairs})\nBlack lines = RNA↔Protein matches')

plt.tight_layout()
plt.show()

# Quantify separation
matched_distances = np.linalg.norm(umap_coords[:n_pairs] - umap_coords[n_pairs:], axis=1)
print(f"\nMatched pair distances in UMAP:")
print(f"  Mean: {matched_distances.mean():.2f}")
print(f"  Median: {np.median(matched_distances):.2f}")
print(f"  (Shorter distances = better integration)")

In [ ]:
# DIAGNOSTIC: Investigate long connecting lines (potential mismatches)
print("="*60)
print("MATCHED PAIR DISTANCE ANALYSIS")
print("="*60)

# Calculate distances for all matched pairs
umap_coords = cca_adata.obsm['X_umap']
n_pairs = len(rna_cca_scores)

pair_distances = []
for i in range(n_pairs):
    rna_umap = umap_coords[i]
    prot_umap = umap_coords[n_pairs + i]
    dist = np.linalg.norm(rna_umap - prot_umap)
    pair_distances.append({
        'pair_idx': i,
        'rna_idx': aligned_rna_indices[i],
        'distance': dist,
        'rna_umap_x': rna_umap[0],
        'rna_umap_y': rna_umap[1],
        'prot_umap_x': prot_umap[0],
        'prot_umap_y': prot_umap[1]
    })

pair_dist_df = pd.DataFrame(pair_distances)

# Statistics
print(f"\nDistance Statistics (all {n_pairs} matched pairs):")
print(f"  Mean:   {pair_dist_df['distance'].mean():.2f}")
print(f"  Median: {pair_dist_df['distance'].median():.2f}")
print(f"  Std:    {pair_dist_df['distance'].std():.2f}")
print(f"  Min:    {pair_dist_df['distance'].min():.2f}")
print(f"  Max:    {pair_dist_df['distance'].max():.2f}")

# Identify outliers (>2 std from mean)
threshold = pair_dist_df['distance'].mean() + 2 * pair_dist_df['distance'].std()
outliers = pair_dist_df[pair_dist_df['distance'] > threshold]
print(f"\nOutliers (distance > {threshold:.2f}): {len(outliers)} pairs ({100*len(outliers)/n_pairs:.1f}%)")

# Plot distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Histogram of distances
axes[0].hist(pair_dist_df['distance'], bins=50, edgecolor='black', alpha=0.7)
axes[0].axvline(threshold, color='red', linestyle='--', label=f'Outlier threshold ({threshold:.1f})')
axes[0].axvline(pair_dist_df['distance'].median(), color='green', linestyle='-', label=f'Median ({pair_dist_df["distance"].median():.1f})')
axes[0].set_xlabel('UMAP Distance')
axes[0].set_ylabel('Count')
axes[0].set_title('Distribution of Matched Pair Distances')
axes[0].legend()

# UMAP colored by match quality
match_quality = 1 - (pair_dist_df['distance'] / pair_dist_df['distance'].max())
axes[1].scatter(pair_dist_df['rna_umap_x'], pair_dist_df['rna_umap_y'], 
                c=match_quality, cmap='RdYlGn', s=5, alpha=0.5)
axes[1].set_xlabel('UMAP 1')
axes[1].set_ylabel('UMAP 2')
axes[1].set_title('RNA cells colored by match quality\n(Green=good, Red=poor)')

# Show worst matches
if len(outliers) > 0:
    worst_10 = pair_dist_df.nlargest(min(10, len(outliers)), 'distance')
    for _, row in worst_10.iterrows():
        axes[2].plot([row['rna_umap_x'], row['prot_umap_x']], 
                     [row['rna_umap_y'], row['prot_umap_y']], 
                     'r-', alpha=0.7, linewidth=1.5)
    axes[2].scatter(worst_10['rna_umap_x'], worst_10['rna_umap_y'], c='blue', s=20, label='RNA', zorder=5)
    axes[2].scatter(worst_10['prot_umap_x'], worst_10['prot_umap_y'], c='orange', s=20, label='Protein', zorder=5)
    axes[2].set_xlabel('UMAP 1')
    axes[2].set_ylabel('UMAP 2')
    axes[2].set_title(f'Top {len(worst_10)} Worst Matches\n(longest lines)')
    axes[2].legend()

plt.tight_layout()
plt.show()

# Recommendations
print("\n" + "="*60)
print("RECOMMENDATIONS")
print("="*60)
if len(outliers) / n_pairs > 0.1:
    print("\n⚠️  HIGH OUTLIER RATE (>10%)")
    print("   Consider:")
    print("   1. Increase filter_prop in MaxFuse to remove more bad matches")
    print("   2. Check if specific cell types are causing mismatches")
    print("   3. Verify shared feature quality between modalities")
elif len(outliers) / n_pairs > 0.05:
    print("\n⚡ MODERATE OUTLIER RATE (5-10%)")
    print("   Some mismatches are normal. Consider filtering worst matches.")
else:
    print("\n✓ LOW OUTLIER RATE (<5%)")
    print("   Integration quality looks good!")

# Optional: Filter out worst matches
print(f"\nTo filter outliers, you could add:")
print(f"   good_matches = pair_dist_df[pair_dist_df['distance'] <= {threshold:.1f}]")

# ADDITIONAL ANALYSIS: Marker expression in outlier pairs
print("\n" + "="*60)
print("MARKER EXPRESSION IN OUTLIER PAIRS")
print("="*60)

if len(outliers) > 0:
    # Get the RNA indices for outliers
    outlier_rna_indices = [aligned_rna_indices[idx] for idx in outliers['pair_idx'].values]
    
    # Key markers to check
    markers_to_check = {
        'CD45/PTPRC': 'PTPRC',
        'CD3/CD3E': 'CD3E', 
        'CD20/MS4A1': 'MS4A1',
        'CD8/CD8A': 'CD8A',
        'CD68': 'CD68',
        'CD31/PECAM1': 'PECAM1',
    }
    
    print(f"\nAnalyzing {len(outliers)} outlier pairs...")
    print(f"\nMean expression in OUTLIERS vs ALL matched pairs:")
    print(f"{'Marker':<15} {'Outliers':>12} {'All Pairs':>12} {'Difference':>12}")
    print("-"*55)
    marker_diffs = []
    for marker_name, rna_gene in markers_to_check.items():
        if rna_gene in rna_adata_lognorm.var_names:
            gene_idx = list(rna_adata_lognorm.var_names).index(rna_gene)
            all_vals = rna_adata_lognorm.X[:, gene_idx]
            if hasattr(all_vals, 'toarray'):
                all_vals = all_vals.toarray().flatten()
            
            # Expression in outlier RNA cells
            outlier_expr = all_vals[outlier_rna_indices].mean()
            # Expression in all matched RNA cells  
            all_matched_expr = all_vals[aligned_rna_indices].mean()
            
            diff = outlier_expr - all_matched_expr
            marker_diffs.append((marker_name, outlier_expr, all_matched_expr, diff))
            print(f"{marker_name:<15} {outlier_expr:>12.3f} {all_matched_expr:>12.3f} {diff:>+12.3f}")
    
    # Find which markers are most different in outliers
    marker_diffs.sort(key=lambda x: abs(x[3]), reverse=True)
    
    print(f"\nOutlier pairs tend to be cells with:")
    for name, out_expr, all_expr, diff in marker_diffs[:3]:
        if abs(diff) > 0.1:
            direction = "HIGHER" if diff > 0 else "LOWER"
            print(f"  - {direction} {name} expression")
    
    # Show protein expression for corresponding matched protein cells
    print(f"\nProtein marker expression in outlier-matched CODEX cells:")
    prot_markers = ['CD45', 'CD3e', 'CD20', 'CD8', 'CD68', 'CD31']
    available_prot = [m for m in prot_markers if m in protein_adata.var_names]
    
    if available_prot:
        # Get protein indices for outliers
        outlier_prot_indices = []
        for pair_idx in outliers['pair_idx'].values:
            rna_idx = aligned_rna_indices[pair_idx]
            if rna_idx in rna_to_prot_matches:
                prot_idx = rna_to_prot_matches[rna_idx][0][0]  # First match
                outlier_prot_indices.append(prot_idx)
        
        print(f"{'Protein':<12} {'Outliers':>12} {'All Matches':>12}")
        print("-"*40)
        for marker in available_prot:
            prot_idx = list(protein_adata.var_names).index(marker)
            outlier_prot_expr = protein_adata.X[outlier_prot_indices, prot_idx].mean()
            all_prot_expr = protein_adata.X[:, prot_idx].mean()
            print(f"{marker:<12} {outlier_prot_expr:>12.3f} {all_prot_expr:>12.3f}")

    # Note: Cluster distribution analysis available after running Harmony (cells below)
    print(f"\nNote: Run Harmony batch correction cells below for cluster analysis of outliers")
else:
    print("No outliers to analyze!")

# SPATIAL VISUALIZATION: Where are the B cell outliers?
print("\n" + "="*60)
print("SPATIAL LOCATION OF OUTLIER PAIRS")
print("="*60)

if len(outliers) > 0:
    # Get spatial coordinates for outlier protein cells
    outlier_spatial = []
    for pair_idx in outliers['pair_idx'].values:
        rna_idx = aligned_rna_indices[pair_idx]
        if rna_idx in rna_to_prot_matches:
            for prot_idx, score in rna_to_prot_matches[rna_idx]:
                outlier_spatial.append({
                    'x': protein_adata.obs['X_centroid'].values[prot_idx],
                    'y': protein_adata.obs['Y_centroid'].values[prot_idx],
                    'prot_idx': prot_idx,
                    'rna_idx': rna_idx,
                    'distance': outliers[outliers['pair_idx'] == pair_idx]['distance'].values[0]
                })
    
    outlier_spatial_df = pd.DataFrame(outlier_spatial)
    
    # Create spatial_df for all matched cells (needed for background plotting)
    all_spatial = []
    for rna_idx, matches in rna_to_prot_matches.items():
        for prot_idx, score in matches:
            all_spatial.append({
                'x': protein_adata.obs['X_centroid'].values[prot_idx],
                'y': protein_adata.obs['Y_centroid'].values[prot_idx],
                'protein_idx': prot_idx,
                'rna_idx': rna_idx
            })
    spatial_df = pd.DataFrame(all_spatial)
    
    # Create figure with multiple views
    fig, axes = plt.subplots(2, 2, figsize=(16, 14))
    
    # Panel 1: All cells with outliers highlighted
    ax = axes[0, 0]
    # Plot all matched cells in gray
    ax.scatter(spatial_df['x'], spatial_df['y'], c='lightgray', s=0.5, alpha=0.3, label='All matched')
    # Highlight outliers in red
    ax.scatter(outlier_spatial_df['x'], outlier_spatial_df['y'], c='red', s=15, alpha=0.8, 
               label=f'B cell outliers (n={len(outlier_spatial_df)})', zorder=5)
    ax.set_xlabel('X (µm)')
    ax.set_ylabel('Y (µm)')
    ax.set_title('Spatial Location of B Cell Outliers\n(Red = long UMAP lines)')
    ax.legend(loc='upper right')
    ax.set_aspect('equal')
    
    # Panel 2: Zoom into region with most outliers
    ax = axes[0, 1]
    # Find the densest region of outliers
    x_center = outlier_spatial_df['x'].median()
    y_center = outlier_spatial_df['y'].median()
    x_range = outlier_spatial_df['x'].std() * 4
    y_range = outlier_spatial_df['y'].std() * 4
    
    # Plot zoomed region
    zoom_mask = (
        (spatial_df['x'] > x_center - x_range) & (spatial_df['x'] < x_center + x_range) &
        (spatial_df['y'] > y_center - y_range) & (spatial_df['y'] < y_center + y_range)
    )
    ax.scatter(spatial_df.loc[zoom_mask, 'x'], spatial_df.loc[zoom_mask, 'y'], 
               c='lightgray', s=2, alpha=0.5)
    
    zoom_outliers = (
        (outlier_spatial_df['x'] > x_center - x_range) & (outlier_spatial_df['x'] < x_center + x_range) &
        (outlier_spatial_df['y'] > y_center - y_range) & (outlier_spatial_df['y'] < y_center + y_range)
    )
    ax.scatter(outlier_spatial_df.loc[zoom_outliers, 'x'], outlier_spatial_df.loc[zoom_outliers, 'y'], 
               c='red', s=30, alpha=0.8, zorder=5)
    ax.set_xlabel('X (µm)')
    ax.set_ylabel('Y (µm)')
    ax.set_title('Zoomed View of Outlier Region')
    ax.set_aspect('equal')
    
    # Panel 3: CD20 expression across tissue with outliers marked
    ax = axes[1, 0]
    if 'CD20' in protein_adata.var_names:
        cd20_idx = list(protein_adata.var_names).index('CD20')
        # Get CD20 expression for all matched protein cells
        cd20_vals = []
        for idx in spatial_df['protein_idx'].values:
            cd20_vals.append(protein_adata.X[idx, cd20_idx])
        
        scatter = ax.scatter(spatial_df['x'], spatial_df['y'], c=cd20_vals, 
                            cmap='Blues', s=0.5, alpha=0.5, vmin=0, vmax=np.percentile(cd20_vals, 95))
        ax.scatter(outlier_spatial_df['x'], outlier_spatial_df['y'], 
                   facecolors='none', edgecolors='red', s=30, linewidth=1.5, label='Outliers', zorder=5)
        plt.colorbar(scatter, ax=ax, label='CD20 (protein)')
        ax.set_xlabel('X (µm)')
        ax.set_ylabel('Y (µm)')
        ax.set_title('CD20 Expression (B cell marker)\nRed circles = outliers')
        ax.set_aspect('equal')
    
    # Panel 4: Outliers colored by match distance
    ax = axes[1, 1]
    scatter = ax.scatter(outlier_spatial_df['x'], outlier_spatial_df['y'], 
                        c=outlier_spatial_df['distance'], cmap='YlOrRd', s=50, alpha=0.8)
    plt.colorbar(scatter, ax=ax, label='UMAP distance')
    ax.set_xlabel('X (µm)')
    ax.set_ylabel('Y (µm)')
    ax.set_title('Outliers Colored by Match Distance\n(Darker = longer UMAP lines)')
    ax.set_aspect('equal')
    
    plt.tight_layout()
    plt.savefig('spleen_outlier_spatial.png', dpi=150, bbox_inches='tight')
    plt.show()
    
    # Summary statistics
    print(f"\nSpatial distribution of outliers:")
    print(f"  X range: {outlier_spatial_df['x'].min():.0f} - {outlier_spatial_df['x'].max():.0f} µm")
    print(f"  Y range: {outlier_spatial_df['y'].min():.0f} - {outlier_spatial_df['y'].max():.0f} µm")
    print(f"  Center: ({outlier_spatial_df['x'].median():.0f}, {outlier_spatial_df['y'].median():.0f}) µm")
    
    # Check if outliers are clustered or dispersed
    from scipy.spatial.distance import pdist
    if len(outlier_spatial_df) > 2:
        outlier_coords = outlier_spatial_df[['x', 'y']].values
        all_coords = spatial_df[['x', 'y']].sample(min(1000, len(spatial_df))).values
        
        outlier_spread = np.median(pdist(outlier_coords))
        all_spread = np.median(pdist(all_coords))
        
        print(f"\n  Median pairwise distance (outliers): {outlier_spread:.0f} µm")
        print(f"  Median pairwise distance (all cells): {all_spread:.0f} µm")
        
        if outlier_spread < all_spread * 0.7:
            print("\n  → Outliers are SPATIALLY CLUSTERED (likely in B cell zones)")
        elif outlier_spread > all_spread * 1.3:
            print("\n  → Outliers are SPATIALLY DISPERSED")
        else:
            print("\n  → Outliers have similar spatial spread as other cells")
    
    print("\nFigure saved to: spleen_outlier_spatial.png")
else:
    print("No outliers to visualize!")

In [ ]:
# # DIAGNOSTIC: Investigate long connecting lines (potential mismatches)
# print("="*60)
# print("MATCHED PAIR DISTANCE ANALYSIS")
# print("="*60)

# # Calculate distances for all matched pairs
# umap_coords = cca_adata.obsm['X_umap']
# n_pairs = len(rna_cca_scores)

# pair_distances = []
# for i in range(n_pairs):
#     rna_umap = umap_coords[i]
#     prot_umap = umap_coords[n_pairs + i]
#     dist = np.linalg.norm(rna_umap - prot_umap)
#     pair_distances.append({
#         'pair_idx': i,
#         'rna_idx': aligned_rna_indices[i],
#         'distance': dist,
#         'rna_umap_x': rna_umap[0],
#         'rna_umap_y': rna_umap[1],
#         'prot_umap_x': prot_umap[0],
#         'prot_umap_y': prot_umap[1]
#     })

# pair_dist_df = pd.DataFrame(pair_distances)

# # Statistics
# print(f"\nDistance Statistics (all {n_pairs} matched pairs):")
# print(f"  Mean:   {pair_dist_df['distance'].mean():.2f}")
# print(f"  Median: {pair_dist_df['distance'].median():.2f}")
# print(f"  Std:    {pair_dist_df['distance'].std():.2f}")
# print(f"  Min:    {pair_dist_df['distance'].min():.2f}")
# print(f"  Max:    {pair_dist_df['distance'].max():.2f}")

# # Identify outliers (>2 std from mean)
# threshold = pair_dist_df['distance'].mean() + 2 * pair_dist_df['distance'].std()
# outliers = pair_dist_df[pair_dist_df['distance'] > threshold]
# print(f"\nOutliers (distance > {threshold:.2f}): {len(outliers)} pairs ({100*len(outliers)/n_pairs:.1f}%)")

# # Plot distribution
# fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# # Histogram of distances
# axes[0].hist(pair_dist_df['distance'], bins=50, edgecolor='black', alpha=0.7)
# axes[0].axvline(threshold, color='red', linestyle='--', label=f'Outlier threshold ({threshold:.1f})')
# axes[0].axvline(pair_dist_df['distance'].median(), color='green', linestyle='-', label=f'Median ({pair_dist_df["distance"].median():.1f})')
# axes[0].set_xlabel('UMAP Distance')
# axes[0].set_ylabel('Count')
# axes[0].set_title('Distribution of Matched Pair Distances')
# axes[0].legend()

# # UMAP colored by match quality
# match_quality = 1 - (pair_dist_df['distance'] / pair_dist_df['distance'].max())
# axes[1].scatter(pair_dist_df['rna_umap_x'], pair_dist_df['rna_umap_y'], 
#                 c=match_quality, cmap='RdYlGn', s=5, alpha=0.5)
# axes[1].set_xlabel('UMAP 1')
# axes[1].set_ylabel('UMAP 2')
# axes[1].set_title('RNA cells colored by match quality\n(Green=good, Red=poor)')

# # Show worst matches
# if len(outliers) > 0:
#     worst_10 = pair_dist_df.nlargest(min(10, len(outliers)), 'distance')
#     for _, row in worst_10.iterrows():
#         axes[2].plot([row['rna_umap_x'], row['prot_umap_x']], 
#                      [row['rna_umap_y'], row['prot_umap_y']], 
#                      'r-', alpha=0.7, linewidth=1.5)
#     axes[2].scatter(worst_10['rna_umap_x'], worst_10['rna_umap_y'], c='blue', s=20, label='RNA', zorder=5)
#     axes[2].scatter(worst_10['prot_umap_x'], worst_10['prot_umap_y'], c='orange', s=20, label='Protein', zorder=5)
#     axes[2].set_xlabel('UMAP 1')
#     axes[2].set_ylabel('UMAP 2')
#     axes[2].set_title(f'Top {len(worst_10)} Worst Matches\n(longest lines)')
#     axes[2].legend()

# plt.tight_layout()
# plt.show()

# # Recommendations
# print("\n" + "="*60)
# print("RECOMMENDATIONS")
# print("="*60)
# if len(outliers) / n_pairs > 0.1:
#     print("\n⚠️  HIGH OUTLIER RATE (>10%)")
#     print("   Consider:")
#     print("   1. Increase filter_prop in MaxFuse to remove more bad matches")
#     print("   2. Check if specific cell types are causing mismatches")
#     print("   3. Verify shared feature quality between modalities")
# elif len(outliers) / n_pairs > 0.05:
#     print("\n⚡ MODERATE OUTLIER RATE (5-10%)")
#     print("   Some mismatches are normal. Consider filtering worst matches.")
# else:
#     print("\n✓ LOW OUTLIER RATE (<5%)")
#     print("   Integration quality looks good!")

# # Optional: Filter out worst matches
# print(f"\nTo filter outliers, you could add:")
# print(f"   good_matches = pair_dist_df[pair_dist_df['distance'] <= {threshold:.1f}]")

# # ADDITIONAL ANALYSIS: Marker expression in outlier pairs
# print("\n" + "="*60)
# print("MARKER EXPRESSION IN OUTLIER PAIRS")
# print("="*60)

# if len(outliers) > 0:
#     # Get the RNA indices for outliers
#     outlier_rna_indices = [aligned_rna_indices[idx] for idx in outliers['pair_idx'].values]
    
#     # Key markers to check
#     markers_to_check = {
#         'CD45/PTPRC': 'PTPRC',
#         'CD3/CD3E': 'CD3E', 
#         'CD20/MS4A1': 'MS4A1',
#         'CD8/CD8A': 'CD8A',
#         'CD68': 'CD68',
#         'CD31/PECAM1': 'PECAM1',
#     }
    
#     print(f"\nAnalyzing {len(outliers)} outlier pairs...")
#     print(f"\nMean expression in OUTLIERS vs ALL matched pairs:")
#     print(f"{'Marker':<15} {'Outliers':>12} {'All Pairs':>12} {'Difference':>12}")
#     print("-"*55)
#     marker_diffs = []
#     for marker_name, rna_gene in markers_to_check.items():
#         if rna_gene in rna_adata_lognorm.var_names:
#             gene_idx = list(rna_adata_lognorm.var_names).index(rna_gene)
#             all_vals = rna_adata_lognorm.X[:, gene_idx]
#             if hasattr(all_vals, 'toarray'):
#                 all_vals = all_vals.toarray().flatten()
            
#             # Expression in outlier RNA cells
#             outlier_expr = all_vals[outlier_rna_indices].mean()
#             # Expression in all matched RNA cells  
#             all_matched_expr = all_vals[aligned_rna_indices].mean()
            
#             diff = outlier_expr - all_matched_expr
#             marker_diffs.append((marker_name, outlier_expr, all_matched_expr, diff))
#             print(f"{marker_name:<15} {outlier_expr:>12.3f} {all_matched_expr:>12.3f} {diff:>+12.3f}")
    
#     # Find which markers are most different in outliers
#     marker_diffs.sort(key=lambda x: abs(x[3]), reverse=True)
    
#     print(f"\nOutlier pairs tend to be cells with:")
#     for name, out_expr, all_expr, diff in marker_diffs[:3]:
#         if abs(diff) > 0.1:
#             direction = "HIGHER" if diff > 0 else "LOWER"
#             print(f"  - {direction} {name} expression")
    
#     # Show protein expression for corresponding matched protein cells
#     print(f"\nProtein marker expression in outlier-matched CODEX cells:")
#     prot_markers = ['CD45', 'CD3e', 'CD20', 'CD8', 'CD68', 'CD31']
#     available_prot = [m for m in prot_markers if m in protein_adata.var_names]
    
#     if available_prot:
#         # Get protein indices for outliers
#         outlier_prot_indices = []
#         for pair_idx in outliers['pair_idx'].values:
#             rna_idx = aligned_rna_indices[pair_idx]
#             if rna_idx in rna_to_prot_matches:
#                 prot_idx = rna_to_prot_matches[rna_idx][0][0]  # First match
#                 outlier_prot_indices.append(prot_idx)
        
#         print(f"{'Protein':<12} {'Outliers':>12} {'All Matches':>12}")
#         print("-"*40)
#         for marker in available_prot:
#             prot_idx = list(protein_adata.var_names).index(marker)
#             outlier_prot_expr = protein_adata.X[outlier_prot_indices, prot_idx].mean()
#             all_prot_expr = protein_adata.X[:, prot_idx].mean()
#             print(f"{marker:<12} {outlier_prot_expr:>12.3f} {all_prot_expr:>12.3f}")

#     # Note: Cluster distribution analysis available after running Harmony (cells below)
#     print(f"\nNote: Run Harmony batch correction cells below for cluster analysis of outliers")
# else:
#     print("No outliers to analyze!")

# # SPATIAL VISUALIZATION: Where are the B cell outliers?
# print("\n" + "="*60)
# print("SPATIAL LOCATION OF OUTLIER PAIRS")
# print("="*60)

# if len(outliers) > 0:
#     # Get spatial coordinates for outlier protein cells
#     outlier_spatial = []
#     for pair_idx in outliers['pair_idx'].values:
#         rna_idx = aligned_rna_indices[pair_idx]
#         if rna_idx in rna_to_prot_matches:
#             for prot_idx, score in rna_to_prot_matches[rna_idx]:
#                 outlier_spatial.append({
#                     'x': protein_adata.obs['X_centroid'].values[prot_idx],
#                     'y': protein_adata.obs['Y_centroid'].values[prot_idx],
#                     'prot_idx': prot_idx,
#                     'rna_idx': rna_idx,
#                     'distance': outliers[outliers['pair_idx'] == pair_idx]['distance'].values[0]
#                 })
    
#     outlier_spatial_df = pd.DataFrame(outlier_spatial)
    
#     # Create spatial_df for all matched cells (needed for background plotting)
#     all_spatial = []
#     for rna_idx, matches in rna_to_prot_matches.items():
#         for prot_idx, score in matches:
#             all_spatial.append({
#                 'x': protein_adata.obs['X_centroid'].values[prot_idx],
#                 'y': protein_adata.obs['Y_centroid'].values[prot_idx],
#                 'protein_idx': prot_idx,
#                 'rna_idx': rna_idx
#             })
#     spatial_df = pd.DataFrame(all_spatial)
    
#     # Create figure with multiple views
#     fig, axes = plt.subplots(2, 2, figsize=(16, 14))
    
#     # Panel 1: All cells with outliers highlighted
#     ax = axes[0, 0]
#     # Plot all matched cells in gray
#     ax.scatter(spatial_df['x'], spatial_df['y'], c='lightgray', s=0.5, alpha=0.3, label='All matched')
#     # Highlight outliers in red
#     ax.scatter(outlier_spatial_df['x'], outlier_spatial_df['y'], c='red', s=15, alpha=0.8, 
#                label=f'B cell outliers (n={len(outlier_spatial_df)})', zorder=5)
#     ax.set_xlabel('X (µm)')
#     ax.set_ylabel('Y (µm)')
#     ax.invert_yaxis()  # Invert Y for spatial plots if needed
#     ax.set_title('Spatial Location of B Cell Outliers\n(Red = long UMAP lines)')
#     ax.legend(loc='upper right')
#     ax.set_aspect('equal')
    
#     # Panel 2: Zoom into region with most outliers
#     ax = axes[0, 1]
#     # Find the densest region of outliers
#     x_center = outlier_spatial_df['x'].median()
#     y_center = outlier_spatial_df['y'].median()
#     x_range = outlier_spatial_df['x'].std() * 4
#     y_range = outlier_spatial_df['y'].std() * 4
    
#     # Plot zoomed region
#     zoom_mask = (
#         (spatial_df['x'] > x_center - x_range) & (spatial_df['x'] < x_center + x_range) &
#         (spatial_df['y'] > y_center - y_range) & (spatial_df['y'] < y_center + y_range)
#     )
#     ax.scatter(spatial_df.loc[zoom_mask, 'x'], spatial_df.loc[zoom_mask, 'y'], 
#                c='lightgray', s=2, alpha=0.5)
    
#     zoom_outliers = (
#         (outlier_spatial_df['x'] > x_center - x_range) & (outlier_spatial_df['x'] < x_center + x_range) &
#         (outlier_spatial_df['y'] > y_center - y_range) & (outlier_spatial_df['y'] < y_center + y_range)
#     )
#     ax.scatter(outlier_spatial_df.loc[zoom_outliers, 'x'], outlier_spatial_df.loc[zoom_outliers, 'y'], 
#                c='red', s=30, alpha=0.8, zorder=5)
#     ax.invert_yaxis()  # Invert Y for spatial plots if needed
#     ax.set_xlabel('X (µm)')
#     ax.set_ylabel('Y (µm)')
#     ax.set_title('Zoomed View of Outlier Region')
#     ax.set_aspect('equal')
    
#     # Panel 3: CD20 expression across tissue with outliers marked
#     ax = axes[1, 0]
#     if 'CD20' in protein_adata.var_names:
#         cd20_idx = list(protein_adata.var_names).index('CD20')
#         # Get CD20 expression for all matched protein cells
#         cd20_vals = []
#         for idx in spatial_df['protein_idx'].values:
#             cd20_vals.append(protein_adata.X[idx, cd20_idx])
        
#         scatter = ax.scatter(spatial_df['x'], spatial_df['y'], c=cd20_vals, 
#                             cmap='Blues', s=0.5, alpha=0.5, vmin=0, vmax=np.percentile(cd20_vals, 95))
#         ax.scatter(outlier_spatial_df['x'], outlier_spatial_df['y'], 
#                    facecolors='none', edgecolors='red', s=30, linewidth=1.5, label='Outliers', zorder=5)
#         plt.colorbar(scatter, ax=ax, label='CD20 (protein)')
#         ax.invert_yaxis()  # Invert Y for spatial plots if needed
#         ax.set_xlabel('X (µm)')
#         ax.set_ylabel('Y (µm)')
#         ax.set_title('CD20 Expression (B cell marker)\nRed circles = outliers')
#         ax.set_aspect('equal')
    
#     # Panel 4: Outliers colored by match distance
#     ax = axes[1, 1]
#     scatter = ax.scatter(outlier_spatial_df['x'], outlier_spatial_df['y'], 
#                         c=outlier_spatial_df['distance'], cmap='YlOrRd', s=50, alpha=0.8)
#     plt.colorbar(scatter, ax=ax, label='UMAP distance')
#     ax.set_xlabel('X (µm)')
#     ax.set_ylabel('Y (µm)')
#     ax.set_title('Outliers Colored by Match Distance\n(Darker = longer UMAP lines)')
#     ax.set_aspect('equal')
#     plt.gca().invert_yaxis()  # Invert Y for spatial plots if needed
#     plt.tight_layout()
#     plt.savefig('spleen_outlier_spatial.png', dpi=150, bbox_inches='tight')
#     plt.show()
    
#     # Summary statistics
#     print(f"\nSpatial distribution of outliers:")
#     print(f"  X range: {outlier_spatial_df['x'].min():.0f} - {outlier_spatial_df['x'].max():.0f} µm")
#     print(f"  Y range: {outlier_spatial_df['y'].min():.0f} - {outlier_spatial_df['y'].max():.0f} µm")
#     print(f"  Center: ({outlier_spatial_df['x'].median():.0f}, {outlier_spatial_df['y'].median():.0f}) µm")
    
#     # Check if outliers are clustered or dispersed
#     from scipy.spatial.distance import pdist
#     if len(outlier_spatial_df) > 2:
#         outlier_coords = outlier_spatial_df[['x', 'y']].values
#         all_coords = spatial_df[['x', 'y']].sample(min(1000, len(spatial_df))).values
        
#         outlier_spread = np.median(pdist(outlier_coords))
#         all_spread = np.median(pdist(all_coords))
        
#         print(f"\n  Median pairwise distance (outliers): {outlier_spread:.0f} µm")
#         print(f"  Median pairwise distance (all cells): {all_spread:.0f} µm")
        
#         if outlier_spread < all_spread * 0.7:
#             print("\n  → Outliers are SPATIALLY CLUSTERED (likely in B cell zones)")
#         elif outlier_spread > all_spread * 1.3:
#             print("\n  → Outliers are SPATIALLY DISPERSED")
#         else:
#             print("\n  → Outliers have similar spatial spread as other cells")
    
#     print("\nFigure saved to: spleen_outlier_spatial.png")
# else:
#     print("No outliers to visualize!")

## Step 10: Cluster and Transfer Labels to Spatial Coordinates

Clustering can use either:
- **Harmony-corrected embedding** (if Step 10b was run) - better modality mixing
- **Original CCA embedding** (if Harmony skipped) - still functional

The cells below automatically detect which embedding is available.

In [ ]:
# Cluster using the best available embedding
# Works with or without Harmony batch correction

# Select embedding: prefer Harmony-corrected if available
if 'cca_adata_harmony' in dir() and hasattr(cca_adata_harmony, 'obsm') and 'X_harmony' in cca_adata_harmony.obsm:
    print("Using Harmony-corrected embedding for clustering")
    clustering_adata = cca_adata_harmony
    use_rep = 'X_harmony'
else:
    print("Using original CCA embedding for clustering (Harmony not run)")
    clustering_adata = cca_adata
    use_rep = None  # Use .X directly

# Build neighbor graph and cluster
sc.pp.neighbors(clustering_adata, n_neighbors=15, use_rep=use_rep)
sc.tl.umap(clustering_adata, min_dist=0.01, spread=2.5)
sc.tl.leiden(clustering_adata, resolution=1.0)

print(f"Found {len(clustering_adata.obs['leiden'].unique())} clusters")

# Visualize clusters
fig, ax = plt.subplots(1, 1, figsize=(8, 6))
embedding_type = "Harmony-corrected" if use_rep == 'X_harmony' else "Original CCA"
sc.pl.umap(clustering_adata, color='leiden', ax=ax, show=False,
           title=f'Leiden Clusters ({embedding_type} embedding)')
plt.tight_layout()
plt.show()

# Check cluster agreement between matched RNA and Protein cells
n_pairs = len(rna_cca_scores)
rna_clusters = clustering_adata.obs['leiden'].values[:n_pairs]
prot_clusters = clustering_adata.obs['leiden'].values[n_pairs:]
agreement = (rna_clusters == prot_clusters).sum() / n_pairs * 100

print(f"\nCluster agreement between matched pairs: {agreement:.1f}%")
if agreement > 80:
    print("  GOOD: High agreement indicates successful integration")
elif agreement > 50:
    print("  MODERATE: Some integration achieved")
else:
    print("  WARNING: Low agreement suggests integration issues")

In [ ]:
# # Cluster using the best available embedding
# # Works with or without Harmony batch correction

# # Select embedding: prefer Harmony-corrected if available
# if 'cca_adata_harmony' in dir() and hasattr(cca_adata_harmony, 'obsm') and 'X_harmony' in cca_adata_harmony.obsm:
#     print("Using Harmony-corrected embedding for clustering")
#     clustering_adata = cca_adata_harmony
#     use_rep = 'X_harmony'
# else:
#     print("Using original CCA embedding for clustering (Harmony not run)")
#     clustering_adata = cca_adata
#     use_rep = None  # Use .X directly

# # Build neighbor graph and cluster
# sc.pp.neighbors(clustering_adata, n_neighbors=15, use_rep=use_rep)
# sc.tl.umap(clustering_adata)
# sc.tl.leiden(clustering_adata, resolution=0.5)

# print(f"Found {len(clustering_adata.obs['leiden'].unique())} clusters")

# # Visualize clusters
# fig, ax = plt.subplots(1, 1, figsize=(8, 6))
# embedding_type = "Harmony-corrected" if use_rep == 'X_harmony' else "Original CCA"
# sc.pl.umap(clustering_adata, color='leiden', ax=ax, show=False,
#            title=f'Leiden Clusters ({embedding_type} embedding)')
# plt.tight_layout()
# plt.show()

# # Check cluster agreement between matched RNA and Protein cells
# n_pairs = len(rna_cca_scores)
# rna_clusters = clustering_adata.obs['leiden'].values[:n_pairs]
# prot_clusters = clustering_adata.obs['leiden'].values[n_pairs:]
# agreement = (rna_clusters == prot_clusters).sum() / n_pairs * 100

# print(f"\nCluster agreement between matched pairs: {agreement:.1f}%")
# if agreement > 80:
#     print("  GOOD: High agreement indicates successful integration")
# elif agreement > 50:
#     print("  MODERATE: Some integration achieved")
# else:
#     print("  WARNING: Low agreement suggests integration issues")

In [ ]:
# Create a mapping from aligned RNA indices to cluster labels
# Uses clusters from the previous cell (works with either Harmony or original CCA)

# Get clusters - prefer clustering_adata if available, otherwise reconstruct
if 'clustering_adata' in dir():
    n_pairs = len(rna_cca_scores)
    rna_clusters = clustering_adata.obs['leiden'].values[:n_pairs]
elif 'cca_adata_harmony' in dir():
    n_pairs = len(rna_cca_scores)
    rna_clusters = cca_adata_harmony.obs['leiden'].values[:n_pairs]
else:
    n_pairs = len(rna_cca_scores)
    rna_clusters = cca_adata.obs['leiden'].values[:n_pairs]

rna_idx_to_cluster = dict(zip(aligned_rna_indices, rna_clusters))
print(f"Mapped {len(rna_idx_to_cluster)} RNA cells to clusters")

# Key markers for cell type identification in spleen
cell_type_markers = [
    'CD3E', 'CD4', 'CD8A',           # T cells
    'MS4A1', 'CD19',                  # B cells
    'CD14', 'CD68', 'CD163',          # Monocytes/Macrophages
    'PECAM1', 'CD34',                 # Endothelial
    'PTPRC',                          # Pan-leukocyte (CD45)
]

# Filter to available markers
available_markers = [m for m in cell_type_markers if m in rna_adata_lognorm.var_names]
print(f"Available markers for annotation: {len(available_markers)}")
print(f"  {available_markers}")

In [ ]:
# Transfer cluster labels to CODEX spatial coordinates
spatial_records = []
for rna_idx, matches in rna_to_prot_matches.items():
    cluster = rna_idx_to_cluster.get(rna_idx)
    if cluster is None:
        continue
    for prot_idx, score in matches:
        spatial_records.append({
            'x': protein_adata.obs['X_centroid'].values[prot_idx],
            'y': protein_adata.obs['Y_centroid'].values[prot_idx],
            'rna_idx': rna_idx,
            'protein_idx': prot_idx,
            'match_score': score,
            'cluster': cluster
        })

spatial_df = pd.DataFrame(spatial_records)
print(f"Spatial mapping: {len(spatial_df)} cells with cluster labels")
print(f"Unique protein cells: {spatial_df['protein_idx'].nunique()}")
print(f"\nCluster distribution:")
print(spatial_df['cluster'].value_counts().sort_index())

# Plot clusters on spatial coordinates
from matplotlib import cm

unique_clusters = sorted(spatial_df['cluster'].unique())
n_clusters = len(unique_clusters)
colors = cm.tab20(np.linspace(0, 1, max(20, n_clusters)))
cluster_colors = {c: colors[i % 20] for i, c in enumerate(unique_clusters)}

# Subsample for plotting if needed
plot_df = spatial_df if len(spatial_df) <= 50000 else spatial_df.sample(n=50000, random_state=42)

plt.figure(figsize=(12, 12))
for cluster in unique_clusters:
    mask = plot_df['cluster'] == cluster
    plt.scatter(plot_df.loc[mask, 'x'], plot_df.loc[mask, 'y'],
                c=[cluster_colors[cluster]], s=5, alpha=0.5, label=f'Cluster {cluster}')

plt.xlabel('X (µm)')
plt.ylabel('Y (µm)')
plt.title('RNA Clusters Transferred to CODEX Spatial Coordinates')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', markerscale=5)
plt.axis('equal')
plt.tight_layout()
plt.show()

## Step 10b: Harmony Batch Correction for Modality Mixing

**Harmony** is specifically designed to remove batch effects while preserving biological signal. 
Here we treat "modality" as a batch variable to force RNA and Protein cells to mix in the embedding.

This is the standard approach used in Seurat v3+ and other popular integration methods.

In [ ]:
# Install and import harmony if needed
try:
    import harmonypy as hm
    print("Harmony loaded successfully")
except ImportError:
    print("Installing harmonypy...")
    import subprocess
    subprocess.check_call(['pip', 'install', 'harmonypy', '-q'])
    import harmonypy as hm
    print("Harmony installed and loaded")

In [ ]:
# Run Harmony on the stacked CCA embedding to mix modalities
# This treats 'modality' as a batch variable and iteratively corrects for it

print("Running Harmony batch correction...")
print("This will force RNA and Protein cells to mix while preserving biological clusters.\n")

# Create a fresh copy for Harmony
cca_adata_harmony = cca_adata.copy()

# Run Harmony - key parameter is theta (diversity clustering penalty)
# Higher theta = more aggressive mixing
harmony_out = hm.run_harmony(
    cca_adata_harmony.X,                    # The CCA embedding
    cca_adata_harmony.obs,                   # Metadata
    'modality',                              # Batch variable to correct
    theta=2.0,                               # Diversity penalty (default=2, higher=more mixing)
    max_iter_harmony=20,                     # Max iterations
    verbose=True
)

# Store corrected embedding
cca_adata_harmony.obsm['X_harmony'] = harmony_out.Z_corr.T

print(f"\nHarmony correction complete!")
print(f"Corrected embedding shape: {cca_adata_harmony.obsm['X_harmony'].shape}")

In [ ]:
# Compute UMAP on Harmony-corrected embedding and compare with original
# Requires running Harmony cells (17-18) first

if 'cca_adata_harmony' not in dir():
    print("Skipping: cca_adata_harmony not found. Run Harmony cells (17-18) first.")
else:
    # Compute UMAP on Harmony-corrected embedding and compare with original
    sc.pp.neighbors(cca_adata_harmony, n_neighbors=15, use_rep='X_harmony')
    sc.tl.umap(cca_adata_harmony, min_dist=0.01, spread=2.5)
    
    # Compare BEFORE and AFTER Harmony
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))
    
    # Row 1: Before Harmony (original CCA stacked)
    sc.pl.umap(cca_adata, color='modality', ax=axes[0, 0], show=False, 
               title='BEFORE Harmony\n(by modality)')
    
    # Add marker expression to cca_adata if not already there
    if 'CD45' not in cca_adata.obs.columns:
        # Add markers from matched data
        n_rna_emb = len(rna_cca_scores)
        marker_map = {'CD45': 'PTPRC', 'CD3e': 'CD3E'}
        for prot_marker, rna_gene in marker_map.items():
            marker_vals = np.zeros(cca_adata.n_obs)
            if rna_gene in rna_adata_lognorm.var_names:
                rna_idx = list(rna_adata_lognorm.var_names).index(rna_gene)
                rna_vals = rna_adata_lognorm.X[:, rna_idx]
                if sparse.issparse(rna_vals):
                    rna_vals = rna_vals.toarray().flatten()
                marker_vals[:n_rna_emb] = rna_vals[aligned_rna_indices]
            if prot_marker in protein_adata.var_names:
                prot_idx = list(protein_adata.var_names).index(prot_marker)
                for i, rna_idx in enumerate(aligned_rna_indices):
                    prot_indices = [m[0] for m in rna_to_prot_matches[rna_idx]]
                    marker_vals[n_rna_emb + i] = protein_adata.X[prot_indices, prot_idx].mean()
            # Z-score per modality
            rna_part = marker_vals[:n_rna_emb]
            prot_part = marker_vals[n_rna_emb:]
            if rna_part.std() > 0:
                marker_vals[:n_rna_emb] = (rna_part - rna_part.mean()) / rna_part.std()
            if prot_part.std() > 0:
                marker_vals[n_rna_emb:] = (prot_part - prot_part.mean()) / prot_part.std()
            cca_adata.obs[prot_marker] = marker_vals
    
    # Copy markers to harmony version
    for marker in ['CD45', 'CD3e']:
        if marker in cca_adata.obs.columns:
            cca_adata_harmony.obs[marker] = cca_adata.obs[marker].values
    
    sc.pl.umap(cca_adata, color='CD45', ax=axes[0, 1], show=False,
               title='BEFORE: CD45 (pan-leukocyte)', cmap='RdBu_r', vmin=-2, vmax=2)
    sc.pl.umap(cca_adata, color='CD3e', ax=axes[0, 2], show=False,
               title='BEFORE: CD3 (T cells)', cmap='RdBu_r', vmin=-2, vmax=2)
    
    # Row 2: After Harmony
    sc.pl.umap(cca_adata_harmony, color='modality', ax=axes[1, 0], show=False,
               title='AFTER Harmony\n(by modality)')
    sc.pl.umap(cca_adata_harmony, color='CD45', ax=axes[1, 1], show=False,
               title='AFTER: CD45 (pan-leukocyte)', cmap='RdBu_r', vmin=-2, vmax=2)
    sc.pl.umap(cca_adata_harmony, color='CD3e', ax=axes[1, 2], show=False,
               title='AFTER: CD3 (T cells)', cmap='RdBu_r', vmin=-2, vmax=2)
    
    plt.suptitle('Harmony Batch Correction: Before vs After', fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()
    
    # Quantify improvement in matched pair distances
    umap_before = cca_adata.obsm['X_umap']
    umap_after = cca_adata_harmony.obsm['X_umap']
    n_pairs = len(rna_cca_scores)
    
    dist_before = np.linalg.norm(umap_before[:n_pairs] - umap_before[n_pairs:], axis=1)
    dist_after = np.linalg.norm(umap_after[:n_pairs] - umap_after[n_pairs:], axis=1)
    
    print(f"\nMatched pair distances (UMAP space):")
    print(f"  BEFORE Harmony - Mean: {dist_before.mean():.2f}, Median: {np.median(dist_before):.2f}")
    print(f"  AFTER Harmony  - Mean: {dist_after.mean():.2f}, Median: {np.median(dist_after):.2f}")
    improvement = (dist_before.mean() - dist_after.mean()) / dist_before.mean() * 100
    print(f"\n  Improvement: {improvement:.1f}% reduction in matched pair distance")

In [ ]:
# Show matched pairs with connecting lines in Harmony-corrected UMAP
# Requires running Harmony cells (17-18) first

if 'cca_adata_harmony' not in dir():
    print("Skipping: cca_adata_harmony not found. Run Harmony cells (17-18) first.")
else:
    from sklearn.neighbors import NearestNeighbors
    
    # Show matched pairs with connecting lines in Harmony-corrected UMAP
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Panel 1: Harmony UMAP with modality coloring
    sc.pl.umap(cca_adata_harmony, color='modality', ax=axes[0], show=False,
               title='Harmony-Corrected UMAP\n(by modality)')
    
    # Panel 2: Same with connecting lines
    umap_harmony = cca_adata_harmony.obsm['X_umap']
    n_pairs = len(rna_cca_scores)
    n_show = 500
    
    np.random.seed(42)
    pair_sample = np.random.choice(n_pairs, min(n_show, n_pairs), replace=False)
    
    for pair_idx in pair_sample:
        rna_pt = umap_harmony[pair_idx]
        prot_pt = umap_harmony[n_pairs + pair_idx]
        axes[1].plot([rna_pt[0], prot_pt[0]], [rna_pt[1], prot_pt[1]], 
                     'black', alpha=0.5, linewidth=1.0)
    
    sc.pl.umap(cca_adata_harmony, color='modality', ax=axes[1], show=False,
               title=f'Matched Pairs Connected\n(n={n_show}, shorter lines = better)')
    
    plt.tight_layout()
    plt.show()
    
    # =================================================================
    # DIAGNOSTIC: Compare kNN accuracy BEFORE vs AFTER Harmony
    # =================================================================
    print("\n" + "="*70)
    print("DIAGNOSTIC: kNN ACCURACY COMPARISON (Before vs After Harmony)")
    print("="*70)
    print("\nHypothesis: Harmony optimizes for distribution mixing, NOT pair preservation")
    print("If kNN accuracy drops after Harmony, it confirms Harmony scrambles correspondences.\n")
    
    # kNN accuracy BEFORE Harmony (on raw CCA stacked embedding)
    print("Computing kNN accuracy on ORIGINAL CCA embedding...")
    knn_before = NearestNeighbors(n_neighbors=50)
    knn_before.fit(cca_adata.X)  # Original stacked CCA
    _, indices_before = knn_before.kneighbors(cca_adata.X)
    
    knn_hits_before_rna = sum(1 for i in range(n_pairs) if (n_pairs + i) in indices_before[i])
    knn_hits_before_prot = sum(1 for i in range(n_pairs) if i in indices_before[n_pairs + i])
    knn_acc_before = (knn_hits_before_rna + knn_hits_before_prot) / (2 * n_pairs) * 100
    
    # kNN accuracy AFTER Harmony
    print("Computing kNN accuracy on HARMONY-corrected embedding...")
    knn_after = NearestNeighbors(n_neighbors=50)
    knn_after.fit(cca_adata_harmony.obsm['X_harmony'])
    _, indices_after = knn_after.kneighbors(cca_adata_harmony.obsm['X_harmony'])
    
    knn_hits_after_rna = sum(1 for i in range(n_pairs) if (n_pairs + i) in indices_after[i])
    knn_hits_after_prot = sum(1 for i in range(n_pairs) if i in indices_after[n_pairs + i])
    knn_acc_after = (knn_hits_after_rna + knn_hits_after_prot) / (2 * n_pairs) * 100
    
    print("\n" + "-"*70)
    print("                          BEFORE Harmony    AFTER Harmony")
    print("-"*70)
    print(f"  kNN accuracy (RNA→Prot):  {knn_hits_before_rna/n_pairs*100:5.1f}%           {knn_hits_after_rna/n_pairs*100:5.1f}%")
    print(f"  kNN accuracy (Prot→RNA):  {knn_hits_before_prot/n_pairs*100:5.1f}%           {knn_hits_after_prot/n_pairs*100:5.1f}%")
    print(f"  Average kNN accuracy:     {knn_acc_before:5.1f}%           {knn_acc_after:5.1f}%")
    print("-"*70)
    
    if knn_acc_before > knn_acc_after:
        print(f"\n⚠️  CONFIRMED: Harmony REDUCED kNN accuracy by {knn_acc_before - knn_acc_after:.1f}%")
        print("    Harmony scrambles individual correspondences while improving distribution overlap.")
    elif knn_acc_before < knn_acc_after:
        print(f"\n✓  Harmony IMPROVED kNN accuracy by {knn_acc_after - knn_acc_before:.1f}%")
    else:
        print("\n    No change in kNN accuracy.")
    
    # Also compute LISI score as a measure of modality mixing
    print("\n" + "="*70)
    print("MODALITY MIXING (why UMAP looks good)")
    print("="*70)
    
    # Simple metric: what fraction of k-neighbors are cross-modality?
    cross_modal_before = 0
    cross_modal_after = 0
    for i in range(2 * n_pairs):
        is_rna = i < n_pairs
        for neighbor in indices_before[i, 1:]:  # Skip self
            neighbor_is_rna = neighbor < n_pairs
            if is_rna != neighbor_is_rna:
                cross_modal_before += 1
        for neighbor in indices_after[i, 1:]:
            neighbor_is_rna = neighbor < n_pairs
            if is_rna != neighbor_is_rna:
                cross_modal_after += 1
    
    cross_frac_before = cross_modal_before / (2 * n_pairs * 49) * 100
    cross_frac_after = cross_modal_after / (2 * n_pairs * 49) * 100
    
    print(f"  Cross-modality neighbors (BEFORE): {cross_frac_before:.1f}% (ideal: 50%)")
    print(f"  Cross-modality neighbors (AFTER):  {cross_frac_after:.1f}% (ideal: 50%)")
    
    if cross_frac_after > cross_frac_before:
        print(f"\n✓  Harmony increased modality mixing by {cross_frac_after - cross_frac_before:.1f}%")
        print("    This is why UMAP looks well-mixed - but at the cost of pair accuracy!")
    
    print("\n" + "="*70)
    print("INTERPRETATION")
    print("="*70)
    print("""
Harmony optimizes a DIFFERENT objective than MaxFuse:
  - MaxFuse: "Cell i in RNA should match cell j in Protein" (pairwise)
  - Harmony: "RNA and Protein distributions should overlap" (global)

A cell can be moved closer to the OTHER modality's cluster while being 
moved AWAY from its specific matched partner.

SOLUTIONS:
1. Use original CCA embedding for clustering (not Harmony-corrected)
2. Skip Harmony entirely - use scanorama or BBKNN if mixing needed
3. Use Harmony only for visualization, keep original for analysis
4. Use MNN-based integration that respects pairing information
""")

## Step 10c: Enhanced Spatial Visualization

Create publication-quality spatial plots with:
- Cell type annotations based on marker expression
- Tissue region highlighting
- Multi-panel comparison views

In [ ]:
# Enhanced Spatial Visualization with Cell Type Annotation
# UPDATED: Pancreas/Lymph Node immune cell types with T cell subsets, CD45+ only
import os
from matplotlib import cm
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches

# Create output directory
os.makedirs('results/3_visualization', exist_ok=True)

# Build spatial dataframe from matching results
spatial_records = []
for rna_idx, matches in rna_to_prot_matches.items():
    cluster = rna_idx_to_cluster.get(rna_idx)
    if cluster is None:
        continue
    for prot_idx, score in matches:
        spatial_records.append({
            'x': protein_adata.obs['X_centroid'].values[prot_idx],
            'y': protein_adata.obs['Y_centroid'].values[prot_idx],
            'rna_idx': rna_idx,
            'protein_idx': prot_idx,
            'match_score': score,
            'cluster': cluster
        })

spatial_df_vis = pd.DataFrame(spatial_records)
print(f"Spatial mapping: {len(spatial_df_vis)} cells with cluster labels")

# =============================================================================
# CD45 (PTPRC) FILTERING - Focus on immune cells only
# =============================================================================
print("\n" + "=" * 60)
print("CD45 (PTPRC) FILTERING - Focus on immune cells")
print("=" * 60)

cd45_gene = 'PTPRC'
if cd45_gene in rna_adata_lognorm.var_names:
    cd45_idx = list(rna_adata_lognorm.var_names).index(cd45_gene)
    cd45_threshold = 0.1  # log-normalized threshold
    
    # Get CD45 expression for each RNA cell
    cd45_by_rna = {}
    for rna_idx in aligned_rna_indices:
        val = rna_adata_lognorm.X[rna_idx, cd45_idx]
        if sparse.issparse(val):
            val = val.toarray().flatten()[0]
        cd45_by_rna[rna_idx] = val
    
    # Add CD45 status to spatial_df
    spatial_df_vis['cd45_expr'] = spatial_df_vis['rna_idx'].map(cd45_by_rna)
    spatial_df_vis['is_cd45_pos'] = spatial_df_vis['cd45_expr'] > cd45_threshold
    
    n_cd45_pos = spatial_df_vis['is_cd45_pos'].sum()
    n_total = len(spatial_df_vis)
    print(f"  Total cells: {n_total:,}")
    print(f"  CD45+ cells: {n_cd45_pos:,} ({100*n_cd45_pos/n_total:.1f}%)")
    
    # Filter to CD45+ only
    spatial_df_vis = spatial_df_vis[spatial_df_vis['is_cd45_pos']].copy()
    print(f"  Proceeding with {len(spatial_df_vis):,} CD45+ cells")
else:
    print(f"  WARNING: {cd45_gene} not found - no filtering applied")

print("=" * 60)

# =============================================================================
# CELL TYPE MARKERS - Pancreas/Lymph Node Immune Cells with T cell subsets
# =============================================================================
cell_type_markers = {
    # T cell subsets (order matters for priority - most specific first)
    'Treg': ['CD3E', 'CD4', 'FOXP3', 'IL2RA'],
    'Exhausted T cell': ['CD3E', 'TOX', 'PDCD1', 'ENTPD1', 'HAVCR2'],
    'CD8+ T cell': ['CD3E', 'CD8A', 'CD8B'],
    'CD4+ T cell': ['CD3E', 'CD4'],
    # Other lymphocytes  
    'B cell': ['MS4A1', 'CD19', 'CD79A', 'CD79B', 'PAX5'],
    'NK cell': ['NCAM1', 'NKG7', 'GNLY', 'GZMB', 'PRF1'],
    # Myeloid cells
    'Macrophage': ['CD68', 'CD163', 'MRC1', 'AIF1', 'CSF1R'],
    'Dendritic cell': ['ITGAX', 'HLA-DRA', 'CD209', 'CLEC9A', 'XCR1'],
    'Neutrophil': ['MPO', 'ITGAM', 'S100A8', 'S100A9', 'CSF3R'],
}

# Color scheme for visualization
cell_type_colors = {
    # T cell subsets (red spectrum)
    'CD4+ T cell': '#E41A1C',      # Red
    'CD8+ T cell': '#FB9A99',      # Light red/pink
    'Treg': '#B2182B',             # Dark red
    'Exhausted T cell': '#D6604D', # Salmon
    # Other lymphocytes
    'B cell': '#377EB8',           # Blue
    'NK cell': '#984EA3',          # Purple
    # Myeloid (green/brown spectrum)
    'Macrophage': '#4DAF4A',       # Green
    'Dendritic cell': '#FF7F00',   # Orange
    'Neutrophil': '#A65628',       # Brown
    'Other': '#999999',            # Gray
}

# Check which markers are available
print("\nMarker availability check:")
for cell_type, markers in cell_type_markers.items():
    available = [m for m in markers if m in rna_adata_lognorm.var_names]
    print(f"  {cell_type}: {len(available)}/{len(markers)} markers available")
    if available:
        print(f"    Found: {available}")

# =============================================================================
# CELL-LEVEL ANNOTATION (not cluster-based) for better T cell subset detection
# =============================================================================
print("\n" + "=" * 60)
print("CELL-LEVEL ANNOTATION - Pancreas/Lymph Node Immune Cells")
print("=" * 60)

def get_marker_score(rna_idx, markers):
    """Get average expression of available markers for a cell."""
    available = [m for m in markers if m in rna_adata_lognorm.var_names]
    if not available:
        return 0
    total = 0
    for marker in available:
        marker_idx = list(rna_adata_lognorm.var_names).index(marker)
        val = rna_adata_lognorm.X[rna_idx, marker_idx]
        if sparse.issparse(val):
            val = val.toarray().flatten()[0]
        total += val
    return total / len(available)

def annotate_cell(rna_idx, markers_dict, threshold=0.3):
    """Assign cell type based on marker expression with priority ordering."""
    # Priority order: most specific T cell subsets first, then other types
    priority_order = [
        'Treg', 'Exhausted T cell', 'CD8+ T cell', 'CD4+ T cell',
        'NK cell', 'B cell', 'Macrophage', 'Dendritic cell', 'Neutrophil'
    ]
    
    for cell_type in priority_order:
        markers = markers_dict[cell_type]
        available = [m for m in markers if m in rna_adata_lognorm.var_names]
        if not available:
            continue
        
        # For T cell subsets, require CD3E expression
        if cell_type in ['Treg', 'Exhausted T cell', 'CD8+ T cell', 'CD4+ T cell']:
            if 'CD3E' in rna_adata_lognorm.var_names:
                cd3_idx = list(rna_adata_lognorm.var_names).index('CD3E')
                cd3_val = rna_adata_lognorm.X[rna_idx, cd3_idx]
                if sparse.issparse(cd3_val):
                    cd3_val = cd3_val.toarray().flatten()[0]
                if cd3_val < threshold:
                    continue  # Not a T cell
        
        # Check if defining markers are expressed
        score = get_marker_score(rna_idx, available)
        if score > threshold:
            return cell_type
    
    return 'Other'

# Annotate each unique RNA cell
rna_idx_to_celltype = {}
unique_rna_in_spatial = spatial_df_vis['rna_idx'].unique()

for rna_idx in unique_rna_in_spatial:
    rna_idx_to_celltype[rna_idx] = annotate_cell(rna_idx, cell_type_markers)

# Add cell type to spatial dataframe
spatial_df_vis['cell_type'] = spatial_df_vis['rna_idx'].map(rna_idx_to_celltype)

# Summary
print("\nCell type distribution (CD45+ immune cells):")
print("-" * 50)
cell_type_counts = spatial_df_vis['cell_type'].value_counts()
for ct, count in cell_type_counts.items():
    pct = 100 * count / len(spatial_df_vis)
    print(f"  {ct:20s}: {count:8,} ({pct:5.1f}%)")
print("-" * 50)
print(f"Total CD45+ cells: {len(spatial_df_vis):,}")

# =============================================================================
# VISUALIZATION
# =============================================================================
fig, axes = plt.subplots(1, 3, figsize=(21, 7))

# Subsample for visualization
np.random.seed(42)
plot_df = spatial_df_vis if len(spatial_df_vis) <= 50000 else spatial_df_vis.sample(n=50000)

# Panel 1: By cluster (original)
unique_clusters = sorted(plot_df['cluster'].unique())
cluster_cmap = cm.tab20(np.linspace(0, 1, max(20, len(unique_clusters))))
for i, cluster in enumerate(unique_clusters):
    mask = plot_df['cluster'] == cluster
    axes[0].scatter(plot_df.loc[mask, 'x'], plot_df.loc[mask, 'y'],
                    c=[cluster_cmap[i % 20]], s=0.5, alpha=0.5)
axes[0].set_xlabel('X (um)', fontsize=12)
axes[0].set_ylabel('Y (um)', fontsize=12)
axes[0].set_title('Clusters from Joint Embedding', fontsize=14)
axes[0].set_aspect('equal')

# Panel 2: By immune cell type
for cell_type in cell_type_colors.keys():
    mask = plot_df['cell_type'] == cell_type
    if mask.sum() > 0:
        axes[1].scatter(plot_df.loc[mask, 'x'], plot_df.loc[mask, 'y'],
                        c=cell_type_colors[cell_type], s=0.5, alpha=0.5, label=cell_type)
axes[1].set_xlabel('X (um)', fontsize=12)
axes[1].set_ylabel('Y (um)', fontsize=12)
axes[1].set_title('Immune Cell Types (CD45+ only)\nPancreas/Lymph Node', fontsize=14)
axes[1].legend(loc='upper right', markerscale=8, fontsize=9)
axes[1].set_aspect('equal')

# Panel 3: Match quality
scatter = axes[2].scatter(plot_df['x'], plot_df['y'],
                          c=plot_df['match_score'], s=0.5, alpha=0.5,
                          cmap='RdYlGn', vmin=0, vmax=1)
axes[2].set_xlabel('X (um)', fontsize=12)
axes[2].set_ylabel('Y (um)', fontsize=12)
axes[2].set_title('Match Confidence Score', fontsize=14)
axes[2].set_aspect('equal')
plt.colorbar(scatter, ax=axes[2], label='Score (higher=better)')

plt.tight_layout()
plt.savefig('results/3_visualization/spatial_overview.png', dpi=150, bbox_inches='tight')
plt.show()

# Also create cluster_annotations dict for compatibility with downstream code
cluster_annotations = {}
for cluster in spatial_df_vis['cluster'].unique():
    # Get most common cell type in this cluster
    cluster_cells = spatial_df_vis[spatial_df_vis['cluster'] == cluster]['cell_type']
    most_common = cluster_cells.mode().iloc[0] if len(cluster_cells) > 0 else 'Other'
    cluster_annotations[cluster] = most_common

print(f"\nCluster annotations (mode of cell types):")
for cluster in sorted(cluster_annotations.keys()):
    print(f"  Cluster {cluster}: {cluster_annotations[cluster]}")

In [ ]:
# Additional cell type visualization and validation
# Since we filtered for CD45+ cells, we skip endothelial detection (CD45-)

print("=" * 60)
print("CELL TYPE VALIDATION")
print("=" * 60)

# Show T cell subset breakdown
t_cell_types = ['CD4+ T cell', 'CD8+ T cell', 'Treg', 'Exhausted T cell']
t_cells = spatial_df_vis[spatial_df_vis['cell_type'].isin(t_cell_types)]
print(f"\nT cell subsets breakdown:")
print(f"  Total T cells: {len(t_cells):,} ({100*len(t_cells)/len(spatial_df_vis):.1f}%)")
for ct in t_cell_types:
    n = (spatial_df_vis['cell_type'] == ct).sum()
    if n > 0:
        print(f"    {ct}: {n:,} ({100*n/len(t_cells):.1f}% of T cells)")

# Myeloid breakdown
myeloid_types = ['Macrophage', 'Dendritic cell', 'Neutrophil']
myeloid = spatial_df_vis[spatial_df_vis['cell_type'].isin(myeloid_types)]
print(f"\nMyeloid cells breakdown:")
print(f"  Total Myeloid: {len(myeloid):,} ({100*len(myeloid)/len(spatial_df_vis):.1f}%)")
for ct in myeloid_types:
    n = (spatial_df_vis['cell_type'] == ct).sum()
    if n > 0:
        print(f"    {ct}: {n:,}")

# Lymphocyte breakdown
lymph_types = ['B cell', 'NK cell'] + t_cell_types
lymph = spatial_df_vis[spatial_df_vis['cell_type'].isin(lymph_types)]
print(f"\nLymphocytes total: {len(lymph):,} ({100*len(lymph)/len(spatial_df_vis):.1f}%)")

# Create final cell_type_with_endo column for compatibility (just copy cell_type)
spatial_df_vis['cell_type_with_endo'] = spatial_df_vis['cell_type']

# Visualize T cell subsets specifically
fig, axes = plt.subplots(1, 2, figsize=(16, 7))

# Panel 1: All immune cell types
plot_df = spatial_df_vis if len(spatial_df_vis) <= 50000 else spatial_df_vis.sample(n=50000)

for cell_type in cell_type_colors.keys():
    mask = plot_df['cell_type'] == cell_type
    if mask.sum() > 0:
        axes[0].scatter(plot_df.loc[mask, 'x'], plot_df.loc[mask, 'y'],
                        c=cell_type_colors[cell_type], s=0.5, alpha=0.5, label=cell_type)
axes[0].set_xlabel('X (um)', fontsize=12)
axes[0].set_ylabel('Y (um)', fontsize=12)
axes[0].set_title('All Immune Cell Types (CD45+)', fontsize=14)
axes[0].legend(loc='upper right', markerscale=8, fontsize=9)
axes[0].set_aspect('equal')

# Panel 2: T cell subsets only
t_cell_plot = plot_df[plot_df['cell_type'].isin(t_cell_types)]
other_plot = plot_df[~plot_df['cell_type'].isin(t_cell_types)]

# Plot non-T cells in gray first
axes[1].scatter(other_plot['x'], other_plot['y'], c='lightgray', s=0.3, alpha=0.3, label='Other immune')

# Then T cell subsets
for ct in t_cell_types:
    mask = t_cell_plot['cell_type'] == ct
    if mask.sum() > 0:
        axes[1].scatter(t_cell_plot.loc[mask, 'x'], t_cell_plot.loc[mask, 'y'],
                        c=cell_type_colors[ct], s=2, alpha=0.7, label=ct)
axes[1].set_xlabel('X (um)', fontsize=12)
axes[1].set_ylabel('Y (um)', fontsize=12)
axes[1].set_title('T Cell Subsets Highlighted', fontsize=14)
axes[1].legend(loc='upper right', markerscale=5, fontsize=9)
axes[1].set_aspect('equal')

plt.tight_layout()
plt.savefig('results/3_visualization/immune_cell_types.png', dpi=150, bbox_inches='tight')
plt.show()

print(f"\nFinal cell type distribution:")
print(spatial_df_vis['cell_type'].value_counts())

## Step 10d: MaxFuse Summary

In [ ]:
# MaxFuse Results Summary
print("="*70)
print("MAXFUSE INTEGRATION SUMMARY")
print("="*70)

# MaxFuse results (from loaded checkpoint)
n_maxfuse_rna = rna_active.shape[0]
n_maxfuse_prot = protein_active.shape[0]
n_maxfuse_matched_rna = len(np.unique(full_matching[0]))
n_maxfuse_matched_prot = len(np.unique(full_matching[1]))

print(f"\n1. DATA SCALE")
print(f"   RNA cells: {n_maxfuse_rna:,}")
print(f"   Protein cells: {n_maxfuse_prot:,}")

print(f"\n2. MATCHING COVERAGE")
print(f"   Total matches: {len(full_matching[0]):,}")
print(f"   Unique RNA cells matched: {n_maxfuse_matched_rna:,} ({100*n_maxfuse_matched_rna/n_maxfuse_rna:.1f}%)")
print(f"   Unique Protein cells matched: {n_maxfuse_matched_prot:,} ({100*n_maxfuse_matched_prot/n_maxfuse_prot:.1f}%)")
print(f"   Mean match score: {np.mean(full_matching[2]):.3f}")

print(f"\n3. EMBEDDING QUALITY")
if 'cancor' in dir():
    print(f"   Mean canonical correlation (top 5): {cancor[:5].mean():.3f}")

print(f"\n4. KEY NOTES")
print("   - MaxFuse uses graph smoothing and pivot-based propagation")
print("   - Scales well to large datasets (>100K cells)")
print("   - For MARIO comparison, see 2_integration.ipynb")

## Step 11: Save Results

Save the integration results for downstream analysis.

In [ ]:
# Save visualization results for downstream analysis
import os
import json as json_module
import pickle

results_dir = 'results/3_visualization'
os.makedirs(results_dir, exist_ok=True)

print("Saving visualization results...")
print("=" * 60)

# 1. Save MaxFuse matching
matching_df = pd.DataFrame({
    'rna_idx': full_matching[0],
    'protein_idx': full_matching[1],
    'score': full_matching[2]
})
matching_df.to_csv(f'{results_dir}/maxfuse_matching.csv', index=False)
print(f"1. MaxFuse matching: {len(matching_df):,} matches")

# 2. Save spatial mapping with cell types
if 'spatial_df_vis' in dir():
    spatial_df_vis.to_csv(f'{results_dir}/spatial_cell_types.csv', index=False)
    print(f"2. Spatial cell types: {len(spatial_df_vis):,} cells")
elif 'spatial_df' in dir():
    spatial_df.to_csv(f'{results_dir}/spatial_cell_types.csv', index=False)
    print(f"2. Spatial clusters: {len(spatial_df):,} cells")

# 3. Save cluster annotations mapping
if 'cluster_annotations' in dir():
    with open(f'{results_dir}/cluster_annotations.json', 'w') as f:
        json_module.dump({str(k): v for k, v in cluster_annotations.items()}, f, indent=2)
    print(f"3. Cluster annotations: {len(cluster_annotations)} clusters")

# 4. Save RNA index to cluster mapping
if 'rna_idx_to_cluster' in dir():
    with open(f'{results_dir}/rna_idx_to_cluster.pkl', 'wb') as f:
        pickle.dump(rna_idx_to_cluster, f)
    print(f"4. RNA-to-cluster mapping: {len(rna_idx_to_cluster):,} cells")

# 5. Save aligned RNA indices (protein matches are in rna_to_prot_matches)
if 'aligned_rna_indices' in dir():
    np.save(f'{results_dir}/aligned_rna_indices.npy', aligned_rna_indices)
    print(f"5. Aligned RNA indices: {len(aligned_rna_indices):,} cells")

# 6. Save CCA embeddings
if 'rna_cca_scores' in dir():
    np.save(f'{results_dir}/rna_cca_scores.npy', rna_cca_scores)
    np.save(f'{results_dir}/prot_cca_scores.npy', prot_cca_scores)
    print(f"6. CCA embeddings: {rna_cca_scores.shape}")

# 7. Save joint embedding AnnData
if 'cca_adata_harmony' in dir() and hasattr(cca_adata_harmony, 'obsm') and 'X_harmony' in cca_adata_harmony.obsm:
    cca_adata_harmony.write(f'{results_dir}/joint_embedding.h5ad')
    print(f"7. Joint embedding (Harmony): {cca_adata_harmony.shape}")
elif 'clustering_adata' in dir():
    clustering_adata.write(f'{results_dir}/joint_embedding.h5ad')
    print(f"7. Joint embedding: {clustering_adata.shape}")
elif 'cca_adata' in dir():
    cca_adata.write(f'{results_dir}/joint_embedding.h5ad')
    print(f"7. Joint embedding: {cca_adata.shape}")

# 8. Save RNA-to-protein matches dictionary
if 'rna_to_prot_matches' in dir():
    with open(f'{results_dir}/rna_to_prot_matches.pkl', 'wb') as f:
        pickle.dump(rna_to_prot_matches, f)
    print(f"8. RNA-to-protein match dict: {len(rna_to_prot_matches):,} RNA cells")

# 9. Save analysis parameters
vis_params = {
    'n_matched_pairs': len(aligned_rna_indices) if 'aligned_rna_indices' in dir() else 0,
    'n_clusters': len(cluster_annotations) if 'cluster_annotations' in dir() else 0,
    'cell_types': list(set(cluster_annotations.values())) if 'cluster_annotations' in dir() else [],
    'n_cca_components': rna_cca_scores.shape[1] if 'rna_cca_scores' in dir() else 0,
}
with open(f'{results_dir}/analysis_params.json', 'w') as f:
    json_module.dump(vis_params, f, indent=2)
print(f"9. Analysis parameters saved")

print("=" * 60)
print(f"\nAll outputs saved to {results_dir}/")
print("\nRun 5_analysis.ipynb for gene expression inference analysis.")